# Predicción del precio de Airbnb Madrid España: Pre-procesamiento y Modelado 

<div class="alert alert-info alert-info"><b><h1>Objetivo</h1></b>
    
**Crear un modelo de regresión lineal con el dataset previamente analizado, asi como utilizar las métricas para interpretar los resultados.**
</div>

## 1. Introducción:  

En el notebook anterior se realizó un análisis detallado de los precios de Airbnb en Madrid con R. Se llevaron a cabo los siguientes pasos: 
* Descripción de Datos.
* Limpieza y Preparación de Datos.
* Visualización de Datos. 

En este notebook, nos enfocaremos en aplicar técnicas de modelado y verificación del modelo para predecir los precios de los listados en Madrid. El objetivo final es desarrollar un modelo predictivo con regresión lineal, que pueda ayudar a sugerir precios para futuros listados de Airbnb en la ciudad.

## 2. Descripción de Variables:  


In [3]:
from IPython.display import Image
Image("/home/neivysg/keepcoding_glovo_bootcamp/Tech-Girls-Glovo/notebooks/variables.png")

FileNotFoundError: No such file or directory: '/home/neivysg/keepcoding_glovo_bootcamp/Tech-Girls-Glovo/notebooks/variables.png'

FileNotFoundError: No such file or directory: '/home/neivysg/keepcoding_glovo_bootcamp/Tech-Girls-Glovo/notebooks/variables.png'

<IPython.core.display.Image object>

---

## 3. Librerías a utilizar

In [4]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import joypy
from joypy import joyplot


# Importar el módulo para la separación en train y test
from sklearn.model_selection import train_test_split

# Importar librerías para el análisis estadístico
from scipy import stats
from scipy.stats import shapiro, ttest_ind, mannwhitneyu
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Importar los módulos para el preprocesado
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


# Importar los módulos para la Regresión
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


# Importar los módulos para los cálculos métricos
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import sklearn.metrics as metrics

# Importar módulo para el pipeline
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'joypy'

---

## 4. Dataset

In [5]:
df_airbnb_clean = pd.read_csv('/home/neivysg/keepcoding_glovo_bootcamp/Tech-Girls-Glovo/data/processed/processed-air-bnb-listings.csv', 
                              sep=';', 
                              encoding='UTF-8')

FileNotFoundError: [Errno 2] No such file or directory: '/home/neivysg/keepcoding_glovo_bootcamp/Tech-Girls-Glovo/data/processed/processed-air-bnb-listings.csv'

In [ ]:
df_airbnb_clean.info()

In [ ]:
df_airbnb_clean.head()

## 5. Análisis General

### 5.1. Verificar nulos y NaM

In [ ]:
df_airbnb_clean.isnull().sum()

In [ ]:
df_airbnb_clean.isna().sum()

### 5.2. Verificar datos duplicados

In [ ]:
df_airbnb_clean[df_airbnb_clean.duplicated()]

**Observaciones:**

   *  Se puede observar que el dataset  tiene valores nulos o NaN en las columnas names y Date.last.review. 
   *  Se puede observar que el data set no tiene valores duplicados.   
   * Se tienen 14 variables objetos y 6 integer.
   * Basado en el Análisis Exploratorio previo (Cuaderno R) se tienen seleccionadas inicialmente para el modelo de predicción los siguientes atributos; sin embargo, previamente se realizará un análisis estadístico preliminar a la selección final o caso base:
      - Atributo objetivo: Room.Price
      - Numéricas: Minimum.nights, Number.of.reviews, Number.of.reviews.per.month, Latitude, Longitude.
      - Categóricas: Neighbourhood, Room.type, Availability_Cat, Review_category, Review_Count_Category, Time_category.
   * Se descartarán los atributos: Room.ID, Name, Host.ID, Date.last.review, Updated.Date, City, Country. 

### 5.3. Eliminar atributos

De acuerdo al analisis exploratorio previo, se eliminaran los atributos: Room.ID, Name, Host.ID, Date.last.review, Updated.Date, City, Country, como primer descarte.

In [ ]:
df_airbnb_clean_vs_1 = df_airbnb_clean.drop(['Room.ID', 'Name', 'Host.ID', 'Date.last.review', 'Updated.Date', 'City', 'Country'], axis=1)

In [ ]:
df_airbnb_clean_vs_1.info()

### 5.4. Verificar tipos de datos

In [ ]:
# Verificar tipos de datos
print(df_airbnb_clean_vs_1.dtypes)

Dado que Latitude, Longitude y Number.of.reviews.per.month están etiquetados como object, necesitamos convertir estas columnas a tipo numérico

In [ ]:
# Convertir columnas a tipo numérico
df_airbnb_clean_vs_1['Latitude'] = df_airbnb_clean_vs_1['Latitude'].str.replace(',', '.').astype(float)
df_airbnb_clean_vs_1['Longitude'] = df_airbnb_clean_vs_1['Longitude'].str.replace(',', '.').astype(float)
df_airbnb_clean_vs_1['Number.of.reviews.per.month'] = df_airbnb_clean_vs_1['Number.of.reviews.per.month'].str.replace(',', '.').astype(float)

In [ ]:
df_airbnb_clean_vs_1.info()

In [ ]:
df_airbnb_clean_vs_1.head()

### 5.5. Descripción estadística y visualización global de las variables.

Para determinar qué variables mantener para el modelo de predicción, necesitamos evaluar su correlación con el precio de la habitación (Room.Price) y considerar otras técnicas estadísticas. Los pasos serán:

1. Sumario estadístico: Numéricas y categóricas.
2. Evaluar la correlación: Ver la correlación de los atributos numéricos con Room.Price.
3. Análisis de varianza: Evaluar variables categóricas con ANOVA.
4. Aplicación de prueba de hipótesis para chequear la distribución normal

#### 5.5.1 Sumario Estadístico:

##### Variables Numéricas

In [ ]:
def estadisticos_num(cont_data):
    #Calcular estadísticas descriptivas, redondeamos y transponemos
    estadisticos = cont_data.describe().round(3).T
    #Añadir la mediana
    estadisticos['median'] = cont_data.median().round(3)
    #Reordenar para que la mediana esté al lado de la media
    estadisticos = estadisticos.iloc[:,[0,1,8,2,3,4,5,6,7]]
    #Pedir que nos devuelva los cálculos realizados
    return(estadisticos)

In [ ]:
# Aplicar la función a las variables numéricas
Num_data = df_airbnb_clean_vs_1.select_dtypes(['float64', 'int64'])
Num_data

In [ ]:
estadisticos_num(Num_data)

**Observaciones:**

* Room.Price: El precio medio de una habitación es de aproximadamente 164, pero la mediana es solo 60, lo que indica que hay algunos valores extremadamente altos (como el máximo de 9999) que están elevando la media. El 75% de las habitaciones cuestan 100 o menos.

* Minimum.nights: La mayoría de las estancias requieren muy pocas noches, con una mediana de solo 2 noches. Sin embargo, el máximo es de 1125 noches, lo que indica que hay algunas que requieren estancias extremadamente largas.

* Number.of.reviews: La mayoría de las habitaciones tienen pocas reseñas, con una mediana de solo 6 reseñas. Sin embargo, la media es de 34.875, lo que indica que hay algunas habitaciones con un número muy alto de reseñas.

* Number.of.reviews.per.month: Similar a Number.of.reviews, la mayoría de las habitaciones tienen pocas reseñas por mes, pero hay algunas habitaciones con un número muy alto de reseñas por mes.

* Rooms.rent.by.the.host: La mayoría de los anfitriones alquilan pocas habitaciones, con una mediana de solo 2 habitaciones. Sin embargo, el máximo es de 244, lo que indica que hay algunos anfitriones que alquilan un gran número de habitaciones.

* Latitude y Longitude: Estas son las coordenadas geográficas de las habitaciones. No hay mucho que analizar aquí sin un contexto geográfico adicional.

En general, parece que hay una gran variabilidad en los datos, con algunos valores extremos en varias columnas. Esto podría afectar a los modelos de machine learning si no se manejan adecuadamente. En el caso de la discretización es una buena estrategia para manejar variables numéricas con muchos valores únicos o con valores extremos. Al convertir "Number.of.reviews" y "Number.of.reviews.per.month" en categorías, reduces la complejidad de los datos y puedes ayudar al modelo a capturar patrones más generales.

##### Variables Categoricas

In [ ]:
cat = df_airbnb_clean_vs_1.select_dtypes('object')
cat

In [ ]:
cat.describe()

**Observaciones:**

* Neighbourhood: Hay 21255 observaciones no nulas, 127 barrios únicos, el barrio más común es "Embajadores", y "Embajadores" aparece 2559 veces.

* Room.type: Hay 21255 observaciones no nulas, 4 tipos de habitaciones únicos, el tipo de habitación más común es "Entire home/apt", y "Entire home/apt" aparece 12704 veces.

* Availability_Cat: Hay 21255 observaciones no nulas, 6 categorías de disponibilidad únicas, la categoría de disponibilidad más común es "181 a 365 días", y "181 a 365 días" aparece 8186 veces.

* Review_category: Hay 21255 observaciones no nulas, 4 categorías de revisión únicas, la categoría de revisión más común es "0-1/mes", y "0-1/mes" aparece 8771 veces.

* Review_Count_Category: Hay 21255 observaciones no nulas, 6 categorías de conteo de revisión únicas, la categoría de conteo de revisión más común es "sin reseñas", y "sin reseñas" aparece 5400 veces.

* Time_category: Hay 21255 observaciones no nulas, 5 categorías de tiempo únicas, la categoría de tiempo más común es "8 semanas - 6 meses", y "8 semanas - 6 meses" aparece 7150 veces.

* Estos resúmenes nos dan una idea de la distribución de las variables categóricas. Para usar estas variables en un modelo de machine learning, se tendrán que codificar en una forma que el modelo pueda entender, como one-hot encoding, ordinal encoding, target encoding, etc.

#### 5.5.2 Evaluar la Correlación de atributos numéricos:

El atributo Room.Price será el target o variable de predicción. Se crearán unos gráficos de dispersión de los atributos en función del atributo Room.Prices para ver su relación.

In [ ]:
## Asignar el índice de la columna que queremos seleccionar
col_idx = 0

## Crear gráficos con cuatro columnas y tres filas
fig, axes = plt.subplots(ncols=3, nrows=3, figsize=(30, 15))

## Recorrer cada subplot y hacemos un scatterplot
for i in range(3):
    for j in range(3):
        if col_idx < len(Num_data.columns):
            col = Num_data.columns[col_idx]
            if np.issubdtype(Num_data[col].dtype, np.number):  # Comprueba si la columna es numerica
                axes[i, j].plot(Num_data[col], Num_data['Room.Price'], 'o', color='tab:blue')
                axes[i, j].set_xlabel(Num_data.columns[col_idx], fontsize=20)
                axes[i, j].set_ylabel('Room.Price', fontsize=20)
                axes[i, j].set_title('R: {:.4f}'.format(Num_data[['Room.Price', col]].corr().iloc[0, 1]), fontsize=20)
                fig.suptitle('Scatterplot of all features', fontsize=30)
            col_idx += 1
        else:
            axes[i, j].axis('off')  # Desactiva el eje si no hay más columnas disponibles

plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajusta el espacio entre subplots
plt.show()

In [ ]:
# Calcular la correlación
correlation_matrix = Num_data.corr()
print(correlation_matrix['Room.Price'].sort_values(ascending=False))

# Crear la máscara
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Visualizar la matriz de correlación
plt.figure(figsize=(12, 8))
ax = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', mask=mask)
ax.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12)
plt.show()

**Observaciones:**

#### 5.5.3 Análisis de Varianza (ANOVA) 

Análisis de Varianza (ANOVA) para Variables Categóricas: 
Evaluemos las variables categóricas (Neighbourhood, Room.type, City, Country, Availability_Cat, Review_category, Review_Count_Category, Time_category) con ANOVA para ver si tienen una relación significativa con Room.Price.

In [ ]:
# Reemplazar los puntos en los nombres de las columnas con guiones bajos
df_airbnb_clean_vs_1.columns = df_airbnb_clean_vs_1.columns.str.replace('.', '_')

In [ ]:
# Realizar ANOVA para cada variable categórica
anova_results = {}

for var in df_airbnb_clean_vs_1.columns:
    if var != 'Room_Price':
        model = ols(f'Room_Price ~ {var}', data=df_airbnb_clean_vs_1).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_results[var] = anova_table

# Mostrar resultados
for var, anova_table in anova_results.items():
    print(f"ANOVA results for {var}:")
    print(anova_table)
    print("\n")

**Observaciones:**
* Room_type: Con un valor F de 68.94 y un valor p extremadamente pequeño (2.33e-44), esto indica que el tipo de habitación tiene un efecto significativo en el precio de la habitación.

* Number_of_reviews: Con un valor F de 119.92 y un valor p extremadamente pequeño (7.82e-28), esto indica que el número de revisiones tiene un efecto significativo en el precio de la habitación.

* Number_of_reviews_per_month: Con un valor F de 177.70 y un valor p extremadamente pequeño (2.24e-40), esto indica que el número de revisiones por mes tiene un efecto significativo en el precio de la habitación.

* Rooms_rent_by_the_host: Con un valor F de 631.93 y un valor p extremadamente pequeño (1.93e-137), esto indica que el número de habitaciones alquiladas por el anfitrión tiene un efecto significativo en el precio de la habitación.

* Latitude y Longitude: Ambas tienen valores F significativos y valores p muy pequeños, lo que indica que la ubicación geográfica de la habitación (latitud y longitud) tiene un efecto significativo en el precio de la habitación.

* Availability_Cat, Review_category, Review_Count_Category, y Time_category: Todas estas variables tienen valores F significativos y valores p muy pequeños, lo que indica que tienen un efecto significativo en el precio de la habitación.

En resumen, todas las variables, excepto Minimum_nights, parecen tener un efecto significativo en el precio de la habitación, según los resultados de ANOVA.

#### 5.5.4 Aplicación de prueba de hipótesis para chequear la distribución normal

In [ ]:
def check_normal_distribution(data):
    for i in data.columns:
        shapiro_test = stats.shapiro(data[i])
        print('La variable', i, 'tiene un p-value= ', shapiro_test.pvalue)
        if shapiro_test.pvalue>0.05:
            print('Se acepta la hipotesis nula, la muestra tiene una distribución normal (Probablemente Gaussiana).\n')
        else:
            print('Se rechaza la hipotesis nula, la muestra no tiene una distribución normal (Probablemente no Gaussiana).\n')

In [ ]:
check_normal_distribution(Num_data)

## 6. Pre-Procesamiento

Una vez realizado el análisis general del dataset limpio y seleccionados los atributos para la predicción del precio, se aplicarán al preprocesamiento de los datos las siguientes normalizaciones: 

**Atributos Categóricos:**
 * Neighbourhood
 * Room.type	
 * Availability_Cat	
 * Review_category y Review_Count_Category: en cuanto a los atributos Number.of.reviews" y "Number.of.reviews.per.month" se observó una gran variabilidad en sus datos, por lo cual fueron convertidas en categóricas en el análisis exploratorio en R. 

**Atributos Numéricos:** 
Dado que las variables numéricas independientes no tienen distribucción normal o gausiona se les aplicara normalización:
 * Outliers: De acuerdo al analisis exploratorio en R, se tienen los siguientes atributos con outliers: Minimum.nights, Rooms.rent.by.the.host, Latitude y Longitude por lo que se le aplicará **RobustScaler**.

In [ ]:
# Se crea una copia del dataframe original con la eliminación de la columna 'Time_category'
df_airbnb_transformed = df_airbnb_clean_vs_1.drop(['Number_of_reviews', 'Number_of_reviews_per_month'], axis=1)

In [ ]:
df_airbnb_transformed.info()

### 6.1. Transformaciones numéricas:

In [ ]:
##Outliers
columns_outliers =['Minimum_nights', 'Rooms_rent_by_the_host', 'Latitude', 'Longitude']
Robust_scaler = preprocessing.RobustScaler().fit(df_airbnb_transformed[columns_outliers])
df_airbnb_transformed[columns_outliers] = Robust_scaler.transform(df_airbnb_transformed[columns_outliers])

#dataset normalizado
df_airbnb_transformed.head()

In [ ]:
### gráfico para ver la mejora

variables = ['Minimum_nights', 'Rooms_rent_by_the_host', 'Latitude', 'Longitude']

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(8, 8))
axs = axs.flatten()

for index, variable in enumerate(variables):
    sns.boxplot(y=variable, data=df_airbnb_transformed, ax=axs[index])

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

A pesar de las transformaciones, no mejoran significativamente los valores de los outliers, por lo que se puede plantear probar un modelo eliminando los outliers.

### 

### 6.2. Transformaciones Categóricas:

Para la transformación de las variables categóricas, existen diferentes tecnicas, sin embargo las mas comunes son una técnica llamada codificación one-hot de sklearn y en Python, puedes usar la función get_dummies() de pandas para hacer esto. Para el proyecto se utilizará Sklearn OneHotEncoder con ColumnTransformer, es más robusto y flexible, especialmente si estás construyendo un pipeline de machine learning que incluye preprocesamiento y modelado. 
Puede manejar nuevas categorías en los datos de prueba que no estaban presentes en los datos de entrenamiento. Además, permite aplicar diferentes transformaciones a diferentes columnas, lo que es útil si también necesitas preprocesar variables numéricas de diferentes maneras.

In [ ]:
# Definir las columnas categóricas que quieres transformar
categorical_features = ['Neighbourhood', 'Room_type', 'Availability_Cat', 'Review_category', 'Review_Count_Category', 'Time_category']

# Crear el OneHotEncoder
encoder = OneHotEncoder(drop='first')

# Ajustar y transformar tus datos
categorical_transformed = encoder.fit_transform(df_airbnb_transformed[categorical_features])

In [ ]:
# Convertir a DataFrame
categorical_transformed = pd.DataFrame(categorical_transformed.toarray(), columns=encoder.get_feature_names_out(categorical_features))

In [ ]:
# Eliminar las columnas categóricas originales de df_airbnb_transformed
df_airbnb_transformed = df_airbnb_transformed.drop(columns=categorical_features)

# Unir el DataFrame transformado con df_airbnb_transformed
df_airbnb_transformed = pd.concat([df_airbnb_transformed, categorical_transformed], axis=1)

In [ ]:
df_airbnb_transformed.info()

In [ ]:
df_airbnb_transformed.head(10)

In [ ]:
df_airbnb_transformed.tail(10)

## 7. Definir Objetivos y atributos independientes

In [ ]:
# Se define la variable dependiente y las independientes
x = df_airbnb_transformed.drop(columns = 'Room_Price', axis=1)
y = df_airbnb_transformed['Room_Price']

In [ ]:
x.head()

In [ ]:
y.head(10)

* Para realizar la división del Dataset: Training y Test (Entrenamineto y prueba), se utilizará 80% para el Train y 20% para el Test.

In [ ]:
#Dividir los datos de entrenamiento y test
#random_state=4
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, train_size=0.8)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

## 8. Modelo

Para el proyecto de predicción de Precios del inmueble con el algortimo de regresión lineal en función de las características que se seleccionaron

### 8.1 Caso base: Regresión lineal

* **Instanciar el modelo**

In [ ]:
#Instanciar
rm = LinearRegression()

#Entrenar
rm.fit(x_train,y_train)

* **Predecir el modelo**

In [ ]:
# Predicción del modelo con los datos de entrenamiento (train data)
y_train_pred_rm = rm.predict(x_train)

In [ ]:
# Predicción del modelo con los datos de prueba (test data)
y_pred_rm = rm.predict(x_test)

In [ ]:
prediccion = pd.DataFrame(y_pred_rm, columns = ['PREDICCION'])
prediccion.head(20)

* **Evaluar el modelo**

Para evaluar cada modelo se calculará:
* El R2, que es una medida de la relación lineal entre X e Y, se interpreta como la proporción de la varianza en la variable dependiente que es predecible a partir de la variable independiente. 
* El MSE es una medida que indica qué tan cerca está la regresión de los puntos observados. Cuanto menor sea el MSE, mejor será el pronóstico.

In [ ]:
def evaluacion(data, predicted): 
    
    r2_square = metrics.r2_score(data, predicted)
    mse = metrics.mean_squared_error(data, predicted)
    errors = mean_absolute_error(data, predicted)
    
    print('R2 : ', r2_square)
    print('MAE (Error Absoluto Medio) : ',  errors)
    
    return {'R2': r2_square, 'MAE': errors}


In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train, y_train_pred_rm )

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test, y_pred_rm)

### 8.2 Caso: Random Forest

In [ ]:
#Instanciar
rfr = RandomForestRegressor(max_depth= 25, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 5, n_estimators= 100)

#Entrenar
rfr.fit(x_train,y_train)

In [ ]:
# Predicción del modelo con los datos de entrenamiento (train data)
y_train_pred_rFr = rfr.predict(x_train)

In [ ]:
# Predicción del modelo con los datos de prueba (test data)
y_pred_rFr = rfr.predict(x_test)

In [ ]:
prediccion_rfr = pd.DataFrame(y_pred_rFr, columns = ['PREDICCION'])
prediccion_rfr .head()

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train, y_train_pred_rFr)

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test, y_pred_rFr)

### 8.3 Caso XGBoost 

XGBoost es una biblioteca optimizada para implementaciones de boosting que es extremadamente eficiente y puede manejar valores atípicos de manera efectiva.

In [ ]:
# Crear el modelo con los hiperparámetros obtenidos
xgbr = xgb.XGBRegressor(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
xgbr.fit(x_train, y_train)

In [ ]:
# Predecir en el conjunto de entrenamiento y prueba
y_train_pred_xgbr = xgbr.predict(x_train)
y_test_pred_xgbr = xgbr.predict(x_test)

In [ ]:
prediccion_xgbr = pd.DataFrame(y_test_pred_xgbr, columns = ['PREDICCION'])
prediccion_xgbr .head()

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train, y_train_pred_xgbr )

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test, y_test_pred_xgbr)

### 8.4 Caso: Descartar las variables Latitud y Longitud

In [ ]:
# Se define la variable dependiente y las independientes
x1 = df_airbnb_transformed.drop(columns = ['Room_Price', 'Latitude', 'Longitude'], axis=1)
y1 = df_airbnb_transformed['Room_Price']

In [ ]:
#Dividir los datos de entrenamiento y test
#random_state=4
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x1, y1, random_state=42, train_size=0.80)
print(x_train_1.shape, x_test_1.shape, y_train_1.shape, y_test_1.shape)

* **Random Forest:**

In [ ]:
#Instanciar
rFr =RandomForestRegressor(max_depth= 25, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 5, n_estimators= 100)

#Entrenar
rfr.fit(x_train_1,y_train_1)

In [ ]:
# Predicción del modelo con los datos de entrenamiento (train data)
y_train_1_pred_rFr = rfr.predict(x_train_1)

In [ ]:
# Predicción del modelo con los datos de prueba (test data)
y_pred_rFr_1 = rfr.predict(x_test_1)

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train, y_train_1_pred_rFr)

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test_1, y_pred_rFr_1)

* **XGBoost:**

In [ ]:
# Crear el modelo con los hiperparámetros obtenidos
xgbr = xgb.XGBRegressor(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
xgbr.fit(x_train_1, y_train_1)

In [ ]:
# Predecir en el conjunto de entrenamiento y prueba
y_train_1_pred_xgbr = xgbr.predict(x_train_1)
y_test_1_pred_xgbr = xgbr.predict(x_test_1)

In [ ]:
prediccion_xgbr_1 = pd.DataFrame(y_test_1_pred_xgbr, columns = ['PREDICCION'])
prediccion_xgbr_1 .head()

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train_1, y_train_1_pred_xgbr)

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test_1, y_test_1_pred_xgbr)

---

### 8.5 Caso: Descartar las variables Neiborhood

In [ ]:
df_airbnb_clean_vs_1.info()

In [ ]:
df_airbnb_no_neighb= df_airbnb_clean_vs_1.drop(['Neighbourhood', 'Number_of_reviews', 'Number_of_reviews_per_month'], axis=1)

In [ ]:
##Transformacion Outliers
columns_outliers =['Minimum_nights', 'Rooms_rent_by_the_host', 'Latitude', 'Longitude']
Robust_scaler = preprocessing.RobustScaler().fit(df_airbnb_no_neighb[columns_outliers])
df_airbnb_no_neighb[columns_outliers] = Robust_scaler.transform(df_airbnb_no_neighb[columns_outliers])

#dataset normalizado
df_airbnb_no_neighb.head()

In [ ]:
# Definir las columnas categóricas que quieres transformar
categorical_features = ['Room_type', 'Availability_Cat', 'Review_category', 'Review_Count_Category', 'Time_category']

# Crear el OneHotEncoder
encoder = OneHotEncoder(drop='first')

# Ajustar y transformar tus datos
categorical_transformed = encoder.fit_transform(df_airbnb_no_neighb[categorical_features])

# Convertir a DataFrame
categorical_transformed = pd.DataFrame(categorical_transformed.toarray(), columns=encoder.get_feature_names_out(categorical_features))
# Eliminar las columnas categóricas originales de df_airbnb_transformed
df_airbnb_no_neighb = df_airbnb_no_neighb.drop(columns=categorical_features)

# Unir el DataFrame transformado con df_airbnb_transformed
df_airbnb_no_neighb = pd.concat([df_airbnb_no_neighb, categorical_transformed], axis=1)

df_airbnb_no_neighb.head(10)

In [ ]:
# Se define la variable dependiente y las independientes
x3 = df_airbnb_no_neighb.drop(columns = 'Room_Price', axis=1)
y3 = df_airbnb_no_neighb['Room_Price']

In [ ]:
#Dividir los datos de entrenamiento y test
#random_state=4
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x3, y3, random_state=42, train_size=0.80)
print(x_train_3.shape, x_test_3.shape, y_train_3.shape, y_test_3.shape)

* **Random Forest:**

In [ ]:
#Instanciar
rfr = RandomForestRegressor(max_depth= 25, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 5, n_estimators= 100)

#Entrenar
rfr.fit(x_train_3,y_train_3)

In [ ]:
# Predicción del modelo con los datos de entrenamiento (train data)
y_train_3_pred_rFr_3 = rfr.predict(x_train_3)

# Predicción del modelo con los datos de prueba (test data)
y_pred_rFr_3 = rfr.predict(x_test_3)

prediccion_rfr_3 = pd.DataFrame(y_pred_rFr_3, columns = ['PREDICCION'])
prediccion_rfr_3 .head()

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train_3, y_train_3_pred_rFr_3)

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test_3, y_pred_rFr_3)

* **XG Boost:**

In [ ]:
# Crear el modelo con los hiperparámetros obtenidos
xgbr = xgb.XGBRegressor(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
xgbr.fit(x_train_3, y_train_3)

In [ ]:
# Predecir en el conjunto de entrenamiento y prueba
y_train_3_pred_xgbr = xgbr.predict(x_train_3)
y_test_pred_3_xgbr = xgbr.predict(x_test_3)

In [ ]:
prediccion_xgbr_3 = pd.DataFrame(y_test_pred_3_xgbr, columns = ['PREDICCION'])
prediccion_xgbr_3.head()

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train_3, y_train_3_pred_xgbr)

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test_3, y_test_pred_3_xgbr)

### 8.6. Caso Eliminar los outliers, previo las transformaciones

* Eliminar los outliers del dataset original

In [ ]:
# Código para eliminar outliers Nª1
numeric_cols = df_airbnb_clean_vs_1.select_dtypes(include=[np.number]).columns
Q1 = df_airbnb_clean_vs_1[numeric_cols].quantile(0.25)
Q3 = df_airbnb_clean_vs_1[numeric_cols].quantile(0.75)
IQR = Q3 - Q1
df_airbnb_clean_no_outliers = df_airbnb_clean_vs_1[~((df_airbnb_clean_vs_1[numeric_cols] < (Q1 - 1.5 * IQR)) | (df_airbnb_clean_vs_1[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
# % de outliers
for k, v in df_airbnb_clean_no_outliers[numeric_cols].items():
    q1 = v.quantile(0.25)
    q3 = v.quantile(0.75)
    irq = q3 - q1
    v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
    perc = np.shape(v_col)[0] * 100.0 / np.shape(df_airbnb_clean_no_outliers)[0]
    print("Column %s outliers = %.2f%%" % (k, perc))

In [ ]:
# Definir el tamaño de la figura
plt.figure(figsize=(8, 8))

# Calcular el número de filas y columnas necesarias para acomodar todos los subplots
num_plots = len(numeric_cols)
num_cols = int(np.ceil(np.sqrt(num_plots)))
num_rows = int(np.ceil(num_plots / num_cols))

# Crear un boxplot para cada columna numérica
for i, col in enumerate(numeric_cols):
    plt.subplot(num_rows, num_cols, i+1)
    sns.boxplot(df_airbnb_clean_vs_1[col])
    plt.tight_layout()

plt.show()

**Observacion**
A pesar de eliminar outliers, se continúa observando un % mínimo.

In [ ]:
df_airbnb_clean_no_outliers.info()

In [ ]:
df_airbnb_clean_no_outliers.isnull().sum()

In [ ]:
df_airbnb_clean_no_outliers.isna().sum()

* **Transformación**

In [ ]:
# Se crea una copia del dataframe original con la eliminación de la columna 'Time_category'
df_airbnb_transf_no_outliers = df_airbnb_clean_no_outliers.drop(['Number_of_reviews', 'Number_of_reviews_per_month'], axis=1)

df_airbnb_transf_no_outliers.info()

In [ ]:
# Power Transformer
numeric_cols = list(df_airbnb_transf_no_outliers._get_numeric_data().columns)
pt = PowerTransformer(method="yeo-johnson")
df_airbnb_transf_no_outliers[numeric_cols] = pt.fit_transform(df_airbnb_transf_no_outliers[numeric_cols])

In [ ]:
df_airbnb_transf_no_outliers.info()

In [ ]:
# Definir las columnas categóricas que quieres transformar
categorical_features = ['Room_type', 'Availability_Cat', 'Review_category', 'Review_Count_Category', 'Time_category', 'Neighbourhood']

# Crear el OneHotEncoder
encoder = OneHotEncoder(drop='first')

# Ajustar y transformar tus datos
categorical_transformed = encoder.fit_transform(df_airbnb_transf_no_outliers[categorical_features])

# Convertir a DataFrame
categorical_transformed = pd.DataFrame(categorical_transformed.toarray(), 
                                       columns=encoder.get_feature_names_out(categorical_features), 
                                       index=df_airbnb_transf_no_outliers.index)  # Mantén el mismo índice

# Eliminar las columnas categóricas originales de df_airbnb_transformed
df_airbnb_transf_no_outliers = df_airbnb_transf_no_outliers.drop(columns=categorical_features)

# Unir el DataFrame transformado con df_airbnb_transformed
df_airbnb_transf_no_outliers = pd.concat([df_airbnb_transf_no_outliers, categorical_transformed], axis=1)

df_airbnb_transf_no_outliers.head(10)

In [ ]:
df_airbnb_transf_no_outliers.info()

* **Variable objetivo (target) y atributos (feature)**

In [ ]:
# Se define la variable dependiente y las independientes
x4 = df_airbnb_transf_no_outliers.drop(columns = 'Room_Price', axis=1)
y4 = df_airbnb_transf_no_outliers['Room_Price']

In [ ]:
x4.info()

In [ ]:
#Dividir los datos de entrenamiento y test
#random_state=4
x_train_4, x_test_4, y_train_4, y_test_4 = train_test_split(x4, y4, random_state=42, train_size=0.8)
print(x_train_4.shape, x_test_4.shape, y_train_4.shape, y_test_4.shape)

In [ ]:
#Instanciar
rfr = RandomForestRegressor(max_depth= 25, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 5, n_estimators= 100)

#Entrenar
rfr.fit(x_train_4,y_train_4)

In [ ]:
# Predicción del modelo con los datos de entrenamiento (train data)
y_train_4_pred_rFr_4 = rfr.predict(x_train_4)

# Predicción del modelo con los datos de prueba (test data)
y_pred_rFr_4 = rfr.predict(x_test_4)

prediccion_rfr_4 = pd.DataFrame(y_pred_rFr_4, columns = ['PREDICCION'])
prediccion_rfr_4 .head()

In [ ]:
# Evaluación del modelo con los datos de entrenamiento (train data)
evaluacion(y_train_4, y_train_4_pred_rFr_4)

In [ ]:
# Evaluación del modelo con los datos de prueba (test data)
evaluacion(y_test_4, y_pred_rFr_4)

---

## 9. Resultados

In [ ]:
resultados = []

# Caso base: Regresión Lineal
resultados.append({'Caso': 'Regresión Lineal', **evaluacion(y_test, y_pred_rm)})

# Caso 2: Random Forest
resultados.append({'Caso': 'Random Forest', **evaluacion(y_test, y_pred_rFr)})

# Caso 3: XGBoost 
resultados.append({'Caso': 'XGBoost', **evaluacion(y_test, y_test_pred_xgbr)})

# Caso 4.1: Random Forest sin coordenadas
# Supongamos que `y_pred_rf_sin_coordenadas` son las predicciones del modelo Random Forest sin coordenadas
resultados.append({'Caso': 'RF sin coordenadas', **evaluacion(y_test_1, y_pred_rFr_1)})

# Caso 4.2: XG Boost sin coordenadas
# Supongamos que `y_pred_rf_sin_coordenadas` son las predicciones del modelo Random Forest sin coordenadas
resultados.append({'Caso': 'XG Boost sin coordenadas', **evaluacion(y_test_1, y_test_1_pred_xgbr)})

# Caso 5.1: Random Forest sin Barrios
# Supongamos que `y_pred_rf_sin_barrios` son las predicciones del modelo Random Forest sin barrios
resultados.append({'Caso': 'RF sin Barrios', **evaluacion(y_test_3, y_pred_rFr_3)})

# Caso 5.2:  XG Boost sin Barrios
# Supongamos que `y_pred_rf_sin_barrios` son las predicciones del modelo Random Forest sin barrios
resultados.append({'Caso': 'XG Boost sin Barrios', **evaluacion(y_test_3,  y_test_pred_3_xgbr)})

# Caso 6: Random Forest sin outliers
# Supongamos que `y_pred_rf_sin_outliers` son las predicciones del modelo Random Forest sin outliers
resultados.append({'Caso': 'RF sin outliers', **evaluacion(y_test_4, y_pred_rFr_4)})

# Convertir la lista en un DataFrame
df_resultados = pd.DataFrame(resultados)

In [ ]:
df_resultados.head(8)

### 9.1. Pipeline

In [ ]:
# Se crea una copia del dataframe original con la eliminación de las columnas 'Number_of_reviews' y 'Number_of_reviews_per_month'
df_airbnb_transformed = df_airbnb_clean_vs_1.drop(['Number_of_reviews', 'Number_of_reviews_per_month'], axis=1)

# Definir las columnas numéricas y categóricas
numeric_features = ['Minimum_nights', 'Rooms_rent_by_the_host', 'Latitude', 'Longitude']
categorical_features = ['Neighbourhood', 'Room_type', 'Availability_Cat', 'Review_category', 'Review_Count_Category', 'Time_category']

# Definir los transformadores para las columnas numéricas y categóricas
numeric_transformer = RobustScaler()
categorical_transformer = OneHotEncoder(drop='first')

# Crear el modelo con los hiperparámetros obtenidos
xgbr = xgb.XGBRegressor(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)

# Combinar los transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Definir la pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgbr)
])

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = df_airbnb_transformed.drop('Room_Price', axis=1)
y = df_airbnb_transformed['Room_Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar la pipeline con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Usar la pipeline para hacer predicciones con los datos de prueba
predictions = pipeline.predict(X_test)

---

In [ ]:
# Supongamos que new_data es el nuevo DataFrame
new_data = pd.DataFrame({
    'Minimum_nights': [3],
    'Rooms_rent_by_the_host': [13],
    'Latitude': [40.40],
    'Longitude': [-3.70175],
    'Neighbourhood': ['Embajadores'],
    'Room_type': ['Entire home/apt'],
    'Availability_Cat': ['Hasta 30 días'],
    'Review_category': ['0-1/mes'],
    'Review_Count_Category': ['11-50 reseñas'],
    'Time_category': ['8 semanas - 6 meses']
})

# Usar la pipeline para hacer predicciones
predictions = pipeline.predict(new_data)

print(predictions)

In [ ]:
# Usar la pipeline para hacer predicciones
predictions = pipeline.predict(new_data)

# Calcular el rango de precios basado en el MAE
lower_bound = int(predictions[0] - 96.87)
upper_bound = int(predictions[0] + 96.87)

# Redondear el precio a dos decimales
predicted_price = round(predictions[0], 2)

print(f"El precio predicho es aproximadamente {predicted_price}, con un rango de {lower_bound} a {upper_bound}.")

## 10. Conclusiones y limitaciones

#### 10.1. **Conclusiones:**

Basado en los resultados se concluye lo siguiente:

* Aunque se normalizaron los outliers, no se observó una gran mejora en los datos. 

* Regresión Lineal: Este modelo tiene el rendimiento más bajo en términos de R^2 y el error absoluto medio (MAE) más alto. Esto sugiere que un modelo lineal puede no ser la mejor opción para este conjunto de datos, posiblemente porque la relación entre las características y la variable objetivo no es lineal.

* Random Forest: Este modelo mejora significativamente el rendimiento en comparación con la regresión lineal. Sin embargo, hay variaciones en el rendimiento dependiendo de las características utilizadas.

* Random Forest sin coordenadas: La eliminación de las coordenadas reduce ligeramente el rendimiento del modelo. Esto sugiere que las coordenadas son características importantes para el modelo.

* Random Forest sin Barrios: Sorprendentemente, la eliminación de los barrios mejora el rendimiento del modelo. Esto podría indicar que la característica de los barrios puede estar introduciendo ruido en el modelo, o que otras características están capturando la misma información de una manera más efectiva.

* Random Forest sin outliers: La eliminación de los outliers reduce el rendimiento del modelo, lo que sugiere que estos puntos de datos pueden contener información valiosa.

* XGBoost: Este modelo tiene el rendimiento más alto en términos de R^2 y el MAE más bajo, lo que sugiere que puede ser la mejor opción para este conjunto de datos.

* El modelo XGBoost parece ser el más preciso de los modelos que hemos probado, con un R^2 de 0.682136 y un MAE de 96.868375. Esto significa que el modelo puede explicar aproximadamente el 68.21% de la variación en los precios y que las predicciones del modelo se desvían del valor real en aproximadamente 96.87 unidades en promedio.

* Se ha utilizado un modelo para predecir el precio de una nueva entrada de datos. El precio predicho es 165.0, con un rango de precios de 68 a 261 basado en el MAE de nuestro modelo.

#### 10.2 **Limitaciones**

* Presencia de valores atípicos: los datos contienen valores atípicos, estos pueden afectar el rendimiento de nuestros modelos. Se podría considerar mejores técnicas para manejar estos valores atípicos, como truncarlos o utlizar otro tipo de transformación. 

* Aunque el modelo XGBoost tiene el mejor rendimiento de los modelos, todavía tiene un error considerable. El MAE de 96.868375 significa que las predicciones del modelo pueden desviarse bastante del valor real.

* El rango de precios que hemos calculado se basa en el MAE de nuestro modelo. Sin embargo, este rango asume que el error es uniforme, lo que puede no ser el caso. Por ejemplo, el modelo puede tener un error mayor para precios más altos que para precios más bajos.

* El rendimiento del modelo puede variar dependiendo de los datos de entrada. Por ejemplo, el modelo puede tener un rendimiento peor para ciertos barrios o tipos de habitaciones. Sería útil analizar el rendimiento del modelo para diferentes subconjuntos de los datos.

* Finalmente, aunque el modelo puede hacer predicciones razonablemente precisas, todavía hay un 31.79% de variación en los precios que el modelo no puede explicar (1 - 0.682136). Esto significa que hay otros factores que afectan al precio que nuestro modelo no está considerando.